# 11 - Avansert nettverk

## 11.1 Sockets

* Den mest vanlige måten for kommunikasjon mellom programmer over nettverk
* Transportlaget i TCP/IP stacken
* Styrt av operativsystemet
 * Program bruker systemkall for å opprette sockets og sende og motta data
* To vanlige typer sockets: TCP og UDP
* Client - Server arkitektur


* Anbefalt lesestoff:
 * https://beej.us/guide/bgnet/html/
 * https://docs.python.org/3/howto/sockets.html

### 11.1.1 Hvordan lage sockets

* Representert med ett socket objekt i Python
* Minner veldig om fil-behandling
 * Basert på Linux, hvor _"alt er en fil"_
* Adresse og protokoll familie:
 * IPv4: _socket.AF_INET_
 * IPv6: _socket.AF_INET6_
* Socket type:
 * TCP: _socket.SOCK_STREAM_
 * UDP: _socket.SOCK_DGRAM_

In [ ]:
import socket
my_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

print(my_socket)


### 11.1.2 Socket client

* Trenger en adresse og en port som skal kobles til
* I Python gjøres DNS automatisk
 * Kan også gjøres manuelt med socket.getaddrinfo()
* _socket.connect()_ tar ett _address_ argument
 * Varierer avhengig av hvilken adresse-familie som brukes
 * For _AF_INET_ er den en tupel: _(host, port)_
* _socket.shutdown()_ for å fortelle andre ende av forbindelse at du kobler ned
* _socket.close()_ for å lukke socketen på vår side

In [1]:
import socket
address = ("www.python.org", 80)

my_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
my_socket.connect(address)

my_socket.shutdown(socket.SHUT_RDWR)
my_socket.close()

__Sending og mottak av data:__
* Original metoder:
 * send() og recv()
* Hjelpefunksjoner:
 * sendall()

In [ ]:
import socket
address = ("python.org", 80)

my_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
my_socket.connect(address)

message = b"GET / HTTP/1.1\r\nHost:python.org\r\n\r\n"
sent = my_socket.send(message)

print(f"Bytes sent: {sent}")
print()

data = my_socket.recv(300)
print(data.decode())

my_socket.shutdown(socket.SHUT_RDWR)
my_socket.close()

__send og recv loops:__
* Send returnerer antallet bytes sendt
 * Programmet er ansvarlig for å verifisere at alt er sendt
 * og eventuelt kalle send() på nytt med resterende data
 * sendall() gjør dette automatisk
* recv( _numbytes_ ) returnerer opp til _numbytes_ med data
 * Programmet må derfor kalle recv til det har mottatt forventet mengde data eller til det ikke er mer data

In [ ]:
import socket
address = ("python.org", 80)

my_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
my_socket.connect(address)

message = b"MYEDATA..."
len_message = len(message)
totalsent = 0

while totalsent < len_message:
    sent = my_socket.send(message[totalsent:])
    totalsent += sent
    
print(f"total sent: {totalsent}")

my_socket.shutdown(socket.SHUT_RDWR)
my_socket.close()

* Litt vanskeligere med recv
* Fire muligheter:
 * Konstant lengde på meldinger
 * Spesielt tegn som indikerer ende på en melding
 * En "header" som sier hvor lang meldingen er (eksempel under)
 * Avslutte forbindelsen for å indikere at all data er sendt

In [ ]:
alldata = b""
bytes_recd = 0

MSGLEN = int(my_socket.recv(4).decode())
while bytes_recd < MSGLEN:
    data = my_socket.recv(1024)
    bytes_recd += len(data)
    alldata += data


### 11.1.3 Socket server

* Åpner en port og lytter på innkommende forbindelser
 * "binder" en socket til en port
* Oppretter en ny socket for hver forbindelse, som brukes for å sende og motta data
* Systemkall:
 * socket.bind()
 * socket.listen()
 * socket.accept()

In [ ]:
import socket
address = ("", 8080)

my_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

my_socket.bind(address)
my_socket.listen(1)
conn, address = my_socket.accept()

__Flere forbindelser (uten parallellisering):__

In [ ]:
import socket
address = ("", 8080)

my_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

my_socket.bind(address)
my_socket.listen()
while True:
    conn, address = my_socket.accept()
    print(address)
    data = conn.recv(1024)
    print(data)
    conn.send(b"Hello!")
    conn.close()

## 11.2 Oppgave: echo client og server

* Lag en socket client:
 * som bruker input() til å lese inn meldinger fra brukeren
 * sender dem til serveren og venter på svar
 * printer svaret


* Lag en socket server:
 * som lytter på en port
 * Når den får en forbindelse, printer den avsender addressen og dataen den mottar
 * "Ekkoer" dataen tilbake igjen


* __Tips:__ bruk _address = ("127.0.0.1", 10000)_
 * dersom du får feilmelding om at porten allerede er i bruk kan du bytte port

## 11.3 smtp

* Simple Mail Transfer Protocol
* Definert i RFC 821 og oppdatert i RFC 5321
* Benyttes for å "pushe" eposter til en server
 * IMAP eller POP3 kan brukes for å hente eposter fra en server
* I Python kan man benytte modulen _smtplib_

In [ ]:
import smtplib

fromaddr = "Rick@example.com"
toaddr = "Morty@example.com"
subject = "Hei!"
body = "Hello world!"
msg = f"From: {fromaddr}\r\nTo: {toaddr}\r\nSubject: {subject}\r\n\r\n{body}"

server = smtplib.SMTP('localhost')
server.set_debuglevel(1)
server.sendmail("Rick@example.com", "Morty@example.com", msg)
server.quit()

## 11.4 Oppgave: Send en mail med python til deg selv

* Husk å inkludere fra-adresse, til-adresse, emne og innhold
* Bonus: lag en interaktiv epost-client, som tar input fra brukeren